# A Movie Recommendation Service
### Source: https://www.codementor.io/@jadianes/building-a-recommender-with-apache-spark-python-example-app-part1-du1083qbw

#### Create a SparkContext configured for local mode

In [2]:
import pyspark
sc = pyspark.SparkContext('local[*]')

#### File download
Small: 100,000 ratings and 2,488 tag applications applied to 8,570 movies by 706 users. Last updated 4/2015.   
Full: 21,000,000 ratings and 470,000 tag applications applied to 27,000 movies by 230,000 users. Last updated 4/2015.

In [3]:
complete_dataset_url = 'http://files.grouplens.org/datasets/movielens/ml-latest.zip'
small_dataset_url = 'http://files.grouplens.org/datasets/movielens/ml-latest-small.zip'

#### Download location(s)

In [4]:
import os
datasets_path = os.path.join('/home/jovyan', 'work')
complete_dataset_path = os.path.join(datasets_path, 'ml-latest.zip')
small_dataset_path = os.path.join(datasets_path, 'ml-latest-small.zip')

#### Getting file(s)

In [5]:
import urllib.request
small_f = urllib.request.urlretrieve (small_dataset_url, small_dataset_path)
complete_f = urllib.request.urlretrieve (complete_dataset_url, complete_dataset_path)

#### Extracting file(s)

In [6]:
import zipfile

with zipfile.ZipFile(small_dataset_path, "r") as z:
    z.extractall(datasets_path)

with zipfile.ZipFile(complete_dataset_path, "r") as z:
    z.extractall(datasets_path)

## Loading and parsing datasets
Now we are ready to read in each of the files and create an RDD consisting of parsed lines. 

Each line in the ratings dataset (ratings.csv) is formatted as: 
+ userId,movieId,rating,timestamp 

Each line in the movies (movies.csv) dataset is formatted as:
+ movieId,title,genres 

The format of these files is uniform and simple, so we can use Python split() to parse their lines once they are loaded into RDDs. Parsing the movies and ratings files yields two RDDs: 
+ For each line in the ratings dataset, we create a tuple of (UserID, MovieID, Rating). We drop the timestamp because we do not need it for this recommender.
+ For each line in the movies dataset, we create a tuple of (MovieID, Title). We drop the genres because we do not use them for this recommender.

#### ratings.csv

In [7]:
small_ratings_file = os.path.join(datasets_path, 'ml-latest-small', 'ratings.csv')
small_ratings_raw_data = sc.textFile(small_ratings_file)
small_ratings_raw_data_header = small_ratings_raw_data.take(1)[0]
# Parse
small_ratings_data = small_ratings_raw_data.filter(lambda line: line!=small_ratings_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (tokens[0],tokens[1],tokens[2])).cache()

print ('There are {} recommendations in the complete dataset'.format(small_ratings_data.count()))
small_ratings_data.take(3)

There are 100836 recommendations in the complete dataset


[('1', '1', '4.0'), ('1', '3', '4.0'), ('1', '6', '4.0')]

#### movies.csv

In [8]:
# Load the small dataset file
small_movies_file = os.path.join(datasets_path, 'ml-latest-small', 'movies.csv')

small_movies_raw_data = sc.textFile(small_movies_file)
small_movies_raw_data_header = small_movies_raw_data.take(1)[0]
# parsing
small_movies_data = small_movies_raw_data.filter(lambda line: line!=small_movies_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (tokens[0],tokens[1])).cache()
small_movies_titles = small_movies_data.map(lambda x: (int(x[0]),x[1]))
print ('There are {} movies in the complete dataset'.format(small_movies_titles.count()))   
small_movies_data.take(3)



There are 9742 movies in the complete dataset


[('1', 'Toy Story (1995)'),
 ('2', 'Jumanji (1995)'),
 ('3', 'Grumpier Old Men (1995)')]

## Collaborative Filtering
In Collaborative filtering we make predictions (filtering) about the interests of a user by collecting preferences or taste information from many users (collaborating). The underlying assumption is that if a user A has the same opinion as a user B on an issue, A is more likely to have B's opinion on a different issue x than to have the opinion on x of a user chosen randomly. 

At first, people rate different items (like videos, images, games). Then, the system makes predictions about a user's rating for an item not rated yet. The new predictions are built upon the existing ratings of other users with similar ratings with the active user. In the image, the system predicts that the user will not like the video.

Spark MLlib library for Machine Learning provides a Collaborative Filtering implementation by using Alternating Least Squares. The implementation in MLlib has the following parameters:

+ numBlocks is the number of blocks used to parallelize computation (set to -1 to auto-configure).
+ rank is the number of latent factors in the model.
+ iterations is the number of iterations to run.
+ lambda specifies the regularization parameter in ALS.
+ implicitPrefs specifies whether to use the explicit feedback ALS variant or one adapted for implicit feedback data.
+ alpha is a parameter applicable to the implicit feedback variant of ALS that governs the baseline confidence in preference observations.

#### Selecting ALS parameters using the small dataset
In order to determine the best ALS parameters, we will use the small dataset. We need first to split it into train, validation, and test datasets.

In [9]:
# source uses see=0L, which is the previous version of python (2.x)
# 0L should be written as 0 from now on
training_RDD, validation_RDD, test_RDD = small_ratings_data.randomSplit([6, 2, 2], seed=0)
validation_for_predict_RDD = validation_RDD.map(lambda x: (x[0], x[1]))
test_for_predict_RDD = test_RDD.map(lambda x: (x[0], x[1]))

#### Training phase

In [10]:
from pyspark.mllib.recommendation import ALS
import math

seed = 5
iterations = 10
regularization_parameter = 0.1
ranks = [4, 8, 12]
errors = [0, 0, 0]
err = 0
tolerance = 0.02

min_error = float('inf')
best_rank = -1
best_iteration = -1
for rank in ranks:
    model = ALS.train(training_RDD, rank, seed=seed, iterations=iterations,
                      lambda_=regularization_parameter)
    predictions = model.predictAll(validation_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
    rates_and_preds = validation_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
    error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    errors[err] = error
    err += 1
    print ('For rank {} the RMSE is {}'.format(rank, error))
    if error < min_error:
        min_error = error
        best_rank = rank

print ('The best model was trained with rank {}'.format(best_rank))

For rank 4 the RMSE is 0.908078105265682
For rank 8 the RMSE is 0.916462973348527
For rank 12 the RMSE is 0.917665030756129
The best model was trained with rank 4


#### Let's have a look at how our predictions look.

In [11]:
predictions.take(3)

[((372, 1084), 3.42419871162954),
 ((4, 1084), 3.866749726695713),
 ((402, 1084), 3.4099577968422152)]

Basically we have the UserID, the MovieID, and the Rating, as we have in our ratings dataset. In this case the predictions third element, the rating for that movie and user, is the predicted by our ALS model.

#### Then we join these with our validation data (the one that includes ratings) and the result looks as follows:

In [12]:
rates_and_preds.take(3)

[((1, 457), (5.0, 4.381060760461434)),
 ((1, 1025), (5.0, 4.705295366590298)),
 ((1, 1089), (5.0, 4.979982471805129))]

To that, we apply a squared difference and the we use the mean() action to get the MSE and apply sqrt.

#### Finally we test the selected model.

In [13]:
model = ALS.train(training_RDD, best_rank, seed=seed, iterations=iterations,
                      lambda_=regularization_parameter)
predictions = model.predictAll(test_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
rates_and_preds = test_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    
print ('For testing data the RMSE is %s' % (error))

For testing data the RMSE is 0.9113780946334407


## Using the complete dataset to build the final model
Due to the limitations of virtual machine, we keep using the small dataset instead of complete dataset

We need first to split it into training and test datasets.

In order to build our recommender model, we will use the complete dataset. Therefore, we need to process it the same way we did with the small dataset.

In [14]:
# Load the complete dataset file
complete_ratings_file = os.path.join(datasets_path, 'ml-latest', 'ratings.csv')
complete_ratings_raw_data = sc.textFile(complete_ratings_file)
complete_ratings_raw_data_header = complete_ratings_raw_data.take(1)[0]

# Parse
complete_ratings_data = complete_ratings_raw_data.filter(lambda line: line!=complete_ratings_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (int(tokens[0]),int(tokens[1]),float(tokens[2]))).cache()
    
print ("There are %s recommendations in the complete dataset" % (complete_ratings_data.count()))


There are 27753444 recommendations in the complete dataset


#### Now we are ready to train the recommender model.

In [15]:
training_RDD, test_RDD = complete_ratings_data.randomSplit([7, 3], seed=0)

complete_model = ALS.train(training_RDD, best_rank, seed=seed, 
                           iterations=iterations, lambda_=regularization_parameter)

Now we test on our testing set.

In [16]:
test_for_predict_RDD = test_RDD.map(lambda x: (x[0], x[1]))

predictions = complete_model.predictAll(test_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
rates_and_preds = test_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    
print ('For testing data the RMSE is %s' % (error))

For testing data the RMSE is 0.8318265262101795


## How to make recommendations
Although we aim at building an online movie recommender, now that we know how to have our recommender model ready, we can give it a try providing some movie recommendations. This will help us coding the recommending engine later on when building the web service, and will explain how to use the model in any other circumstances.

When using collaborative filtering, getting recommendations is not as simple as predicting for the new entries using a previously generated model. Instead, we need to train again the model but including the new user preferences in order to compare them with other users in the dataset. That is, the recommender needs to be trained every time we have new user ratings (although a single model can be used by multiple users of course!). This makes the process expensive, and it is one of the reasons why scalability is a problem (and Spark a solution!). Once we have our model trained, we can reuse it to obtain top recomendations for a given user or an individual rating for a particular movie. These are less costly operations than training the model itself.

In [17]:
complete_movies_file = os.path.join(datasets_path, 'ml-latest', 'movies.csv')
complete_movies_raw_data = sc.textFile(complete_movies_file)
complete_movies_raw_data_header = complete_movies_raw_data.take(1)[0]

# Parse
complete_movies_data = complete_movies_raw_data.filter(lambda line: line!=complete_movies_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (int(tokens[0]),tokens[1],tokens[2])).cache()

complete_movies_titles = complete_movies_data.map(lambda x: (int(x[0]),x[1]))
    
print ("There are %s movies in the complete dataset" % (complete_movies_titles.count()))

There are 58098 movies in the complete dataset


Another thing we want to do, is give recommendations of movies with a certain minimum number of ratings. For that, we need to count the number of ratings per movie.

In [18]:
def get_counts_and_averages(ID_and_ratings_tuple):
    nratings = len(ID_and_ratings_tuple[1])
    return ID_and_ratings_tuple[0], (nratings, float(sum(x for x in ID_and_ratings_tuple[1]))/nratings)

movie_ID_with_ratings_RDD = (complete_ratings_data.map(lambda x: (x[1], x[2])).groupByKey())
movie_ID_with_avg_ratings_RDD = movie_ID_with_ratings_RDD.map(get_counts_and_averages)
movie_rating_counts_RDD = movie_ID_with_avg_ratings_RDD.map(lambda x: (x[0], x[1][0]))

### Adding new user ratings
Now we need to rate some movies for the new user. We will put them in a new RDD and we will use the user ID 0, that is not assigned in the MovieLens dataset. Check the dataset movies file for ID to Tittle assignment (so you know what movies are you actually rating).

In [19]:
new_user_ID = 0

# The format of each line is (userID, movieID, rating)


# My Ratings

# new_user_ratings = [
#      (0,7477,1), # Eye See You (D-Tox) (2002)
#      (0,5064,1), # The Count of Monte Cristo (2002)
#      (0,5080,3), # Slackers (2002)
#      (0,5083,2), # Rare Birds (2001)
#      (0,379,1), # Timecop (1994)
#      (0,5096,3), # Baby's Day Out (1994)
#      (0,148238,2), # A Very Murray Christmas (2015)
#      (0,148709,3), # Mojave (2015)
#      (0,3483,3), # Road to El Dorado, The (2000)
#      (0,3513,4) # Rules of Engagement (2000)
#    ]

# My Friend's Ratings

new_user_ratings = [
     (0,7477,5), # Eye See You (D-Tox) (2002)
     (0,5064,4), # The Count of Monte Cristo (2002)
     (0,5080,1), # Slackers (2002)
     (0,5083,4), # Rare Birds (2001)
     (0,379,2), # Timecop (1994)
     (0,5096,3), # Baby's Day Out (1994)
     (0,148238,2), # A Very Murray Christmas (2015)
     (0,148709,4), # Mojave (2015)
     (0,3483,5), # Road to El Dorado, The (2000)
     (0,3513,3) # Rules of Engagement (2000)

   ]

new_user_ratings_RDD = sc.parallelize(new_user_ratings)
print ('New user ratings: {}'.format(new_user_ratings_RDD.take(10)))

New user ratings: [(0, 7477, 5), (0, 5064, 4), (0, 5080, 1), (0, 5083, 4), (0, 379, 2), (0, 5096, 3), (0, 148238, 2), (0, 148709, 4), (0, 3483, 5), (0, 3513, 3)]


Now we add them to the data we will use to train our recommender model. We use Spark's union() transformation for this.

In [20]:
complete_data_with_new_ratings_RDD = complete_ratings_data.union(new_user_ratings_RDD)

And finally we train the ALS model using all the parameters we selected before (when using the small dataset).

In [21]:
from time import time

t0 = time()
new_ratings_model = ALS.train(complete_data_with_new_ratings_RDD, best_rank, seed=seed, 
                              iterations=iterations, lambda_=regularization_parameter)
tt = time() - t0

print ("New model trained in %s seconds" % round(tt,3))

New model trained in 150.85 seconds


## Getting top recommendations
Let's now get some recommendations! For that we will get an RDD with all the movies the new user hasn't rated yet. We will them together with the model to predict ratings.

In [22]:
new_user_ratings_ids = map(lambda x: x[1], new_user_ratings) # get just movie IDs
# keep just those not on the ID list (thanks Lei Li for spotting the error!)
new_user_unrated_movies_RDD = (complete_movies_data.filter(lambda x: x[0] not in new_user_ratings_ids).map(lambda x: (new_user_ID, x[0])))

# Use the input RDD, new_user_unrated_movies_RDD, with new_ratings_model.predictAll() to predict new ratings for the movies
new_user_recommendations_RDD = new_ratings_model.predictAll(new_user_unrated_movies_RDD)

We have our recommendations ready. Now we can print out the 25 movies with the highest predicted ratings. And join them with the movies RDD to get the titles, and ratings count in order to get movies with a minimum number of counts. First we will do the join and see what does the result looks like.

In [23]:
# Transform new_user_recommendations_RDD into pairs of the form (Movie ID, Predicted Rating)
new_user_recommendations_rating_RDD = new_user_recommendations_RDD.map(lambda x: (x.product, x.rating))
new_user_recommendations_rating_title_and_count_RDD = \
    new_user_recommendations_rating_RDD.join(complete_movies_titles).join(movie_rating_counts_RDD)
new_user_recommendations_rating_title_and_count_RDD.take(3)

[(6216,
  ((4.556600405817434, 'Nowhere in Africa (Nirgendwo in Afrika) (2001)'),
   717)),
 (124320, ((4.732577932571873, 'Once a Thief (1965)'), 1)),
 (83916, ((1.8965047144638127, 'Blues in the Night (1941)'), 9))]

So we need to flat this down a bit in order to have (Title, Rating, Ratings Count).

In [24]:
new_user_recommendations_rating_title_and_count_RDD = \
    new_user_recommendations_rating_title_and_count_RDD.map(lambda r: (r[1][0][1], r[1][0][0], r[1][1]))

Finally, get the highest rated recommendations for the new user, filtering out movies with less than 25 ratings.

In [25]:
# top_movies = new_user_recommendations_rating_title_and_count_RDD.filter(lambda r: r[2]>=25).takeOrdered(15, key=lambda x: -x[1])

# print ('TOP 15 recommended movies (with more than 25 reviews):\n%s' %
#         '\n'.join(map(str, top_movies)))

top_movies = new_user_recommendations_rating_title_and_count_RDD.filter(lambda r: r[2]>=100).takeOrdered(15, key=lambda x: -x[1])

print ('TOP 15 recommended movies (with more than 100 reviews):\n%s' %
        '\n'.join(map(str, top_movies)))

TOP 15 recommended movies (with more than 100 reviews):
('Anne of Green Gables: The Sequel (a.k.a. Anne of Avonlea) (1987)', 5.335650927546585, 342)
('North & South (2004)', 5.310135356892424, 389)
('Anne of Green Gables (1985)', 5.25009056091133, 706)
('Pride and Prejudice (1995)', 5.199110857850529, 2919)
('"Sound of Music', 5.022057410686416, 17154)
('"Civil War', 4.979776328784126, 431)
('Wild China (2008)', 4.960534393596472, 105)
('Emma (2009)', 4.955623211434325, 385)
("Schindler's List (1993)", 4.953946402634649, 71516)
('Sense and Sensibility (1995)', 4.949351577356179, 24552)
('Jane Eyre (2006)', 4.930973659948204, 327)
("It's a Wonderful Life (1946)", 4.929851651350551, 17770)
('My Fair Lady (1964)', 4.90786584177767, 12089)
('Persuasion (2007)', 4.90078438594999, 349)
('Hidden Figures (2016)', 4.896389604611755, 2647)


## Getting individual ratings
Another useful usecase is getting the predicted rating for a particular movie for a given user. The process is similar to the previous retreival of top recommendations but, instead of using predcitAll with every single movie the user hasn't rated yet, we will just pass the method a single entry with the movie we want to predict the rating for.

In [26]:
my_movie = sc.parallelize([(0, 500)]) # Quiz Show (1994)
individual_movie_rating_RDD = new_ratings_model.predictAll(new_user_unrated_movies_RDD)
individual_movie_rating_RDD.take(1)

[Rating(user=0, product=116688, rating=1.423140448961824)]

## Genre and other fields
We haven't used the genre and timestamp fields in order to simplify the transformations and the whole tutorial. Incorporating them doesn't represent any problem. A good use could be filtering recommendations by any of them (e.g. recommendations by genre, or recent recommendations) like we have done with the minimum number of ratings.

## Conclusion
Spark's MLlib library provides scalable data analytics through a rich set of methods. Its Alternating Least Squares implementation for Collaborative Filtering is one that fits perfectly in a recommendation engine. Due to its very nature, collaborative filtering is a costly procedure since requires updating its model when new user preferences arrive. Therefore, having a distributed computation engine such as Spark to perform model computation is a must in any real-world recommendation engine like the one we have built here.

Through this tutorial we have described how to build a model using Spark, how to perform some parameter selection using a reduced dataset, and how to update the model every time that new user preferences come in. Additionally, we have explained how the recommender is used in different situations and how its results are joined with product metadata (e.g. movie titles) in order to present its results in a proper way.

## Results and Appendix

### Apply two new users (i.e., yourself, and a friend/family member) with each person’s ratings of 10 movies (use the section, Adding new user ratings). You need to run one user at a time.

#### My Ratings
new_user_ratings = [    
     (0,7477,1), # Eye See You (D-Tox) (2002)    
     (0,5064,1), # The Count of Monte Cristo (2002)    
     (0,5080,3), # Slackers (2002)    
     (0,5083,2), # Rare Birds (2001)    
     (0,379,1), # Timecop (1994)    
     (0,5096,3), # Baby's Day Out (1994)    
     (0,148238,2), # A Very Murray Christmas (2015)    
     (0,148709,3), # Mojave (2015)    
     (0,3483,3), # Road to El Dorado, The (2000)    
     (0,3513,4) # Rules of Engagement (2000)    
   ]

#### My Friend's Ratings
new_user_ratings = [    
     (0,7477,5), # Eye See You (D-Tox) (2002)    
     (0,5064,4), # The Count of Monte Cristo (2002)     
     (0,5080,1), # Slackers (2002)     
     (0,5083,4), # Rare Birds (2001)      
     (0,379,2), # Timecop (1994)      
     (0,5096,3), # Baby's Day Out (1994)            
     (0,148238,2), # A Very Murray Christmas (2015)          
     (0,148709,4), # Mojave (2015)       
     (0,3483,5), # Road to El Dorado, The (2000)     
     (0,3513,3) # Rules of Engagement (2000)     

   ]

### Generate top 15 recommended movies for each user
Scenario 1 - FULL dataset, filtering out movies with less than 25 ratings (meaning 25 or more ratings)      
Scenario 2 - FULL dataset, filtering out movies with less than 100 ratings (meaning 100 or more ratings)

#### User 1 - Scenario 1

TOP 15 recommended movies (with more than 25 reviews):    
('Loose Change 9/11: An American Coup (2009)', 2.9481489449099487, 46)    
('Jimmy Carr: Comedian (2007)', 2.929652793856018, 30)     
('Jim Jefferies: Contraband (2008)', 2.8668544147203816, 31)     
('Tom Segura: Completely Normal (2014)', 2.8598621319599564, 33)       
('Cosmos: A Spacetime Odissey', 2.858895879752192, 37)           
('Jimmy Carr: Being Funny (2011)', 2.8566759705250915, 25)            
('Firebase (2017)', 2.850174269700494, 29)        
('Louis C.K.: One Night Stand (2005)', 2.839191099963582, 38)          
('The Lost Room (2006)', 2.8359560302691724, 280)         
('Jim Jefferies: Alcoholocaust (2010)', 2.8220375209151563, 53)            
('Daniel Tosh: Happy Thoughts (2011)', 2.817290853546212, 27)         
('Jimmy Carr: Telling Jokes (2009)', 2.808957177335726, 37)            
('Perfectos desconocidos (2017)', 2.8064162374641786, 35)            
('DMB (2000)', 2.805681682938827, 29)            
('"Story of Luke', 2.8011097627490527, 31)           


#### User 1 - Scenario 2

TOP 15 recommended movies (with more than 100 reviews):              
('The Lost Room (2006)', 2.8359560302691724, 280)                
('Black Mirror: White Christmas (2014)', 2.7906121293517714, 1074)               
('Sherlock - A Study in Pink (2010)', 2.7589584333082584, 213)              
('Band of Brothers (2001)', 2.755669305103818, 984)              
('Law Abiding Citizen (2009)', 2.747497605442518, 2570)              
('"Matrix', 2.7456716299129305, 84545)              
('Black Mirror', 2.7374457472060243, 180)              
('"Shawshank Redemption', 2.735032725984513, 97999)            
('"Dark Knight', 2.7338810864344367, 44741)             
('Saw (2003)', 2.7314120779550466, 674)                   
('Limitless (2011)', 2.7199078882138386, 9884)             
('"Boondock Saints', 2.7151981868757353, 11214)              
('Avengers: Infinity War - Part I (2018)', 2.7146535995028804, 2668)              
('Gladiator (2000)', 2.707570786796058, 48666)             
('Inception (2010)', 2.7033616454658347, 41475)                

#### User 2 - Scenario 1

TOP 15 recommended movies (with more than 25 reviews):                  
('"Very Potter Sequel', 5.402075474560762, 35)                 
('Sense & Sensibility (2008)', 5.3776839046314455, 69)                
('Anne of Green Gables: The Sequel (a.k.a. Anne of Avonlea) (1987)', 5.335650927546585, 342)                 
('Cranford (2007)', 5.323038303364424, 35)               
('North & South (2004)', 5.310135356892424, 389)                
('Drishyam (2013)', 5.255280979471657, 37)                 
('Anne of Green Gables (1985)', 5.25009056091133, 706)                
('Pride and Prejudice (1995)', 5.199110857850529, 2919)                 
('Murder on the Orient Express (2010)', 5.182371057817914, 29)                  
('I Can Only Imagine (2018)', 5.180623692610432, 30)                    
('Winter in Prostokvashino (1984)', 5.1467791284786335, 67)                    
('Boys (2014)', 5.141713053618693, 96)                  
('Little Dorrit (2008)', 5.132472734383583, 55)                       
("Won't You Be My Neighbor? (2018)", 5.0994357608513745, 83)                    
('The Case for Christ (2017)', 5.09381497368954, 33)                          

#### User 2 - Scenario 2

TOP 15 recommended movies (with more than 100 reviews):                           
('Anne of Green Gables: The Sequel (a.k.a. Anne of Avonlea) (1987)', 5.335650927546585, 342)                     
('North & South (2004)', 5.310135356892424, 389)                  
('Anne of Green Gables (1985)', 5.25009056091133, 706)                         
('Pride and Prejudice (1995)', 5.199110857850529, 2919)                       
('"Sound of Music', 5.022057410686416, 17154)                   
('"Civil War', 4.979776328784126, 431)                      
('Wild China (2008)', 4.960534393596472, 105)                       
('Emma (2009)', 4.955623211434325, 385)                
("Schindler's List (1993)", 4.953946402634649, 71516)                    
('Sense and Sensibility (1995)', 4.949351577356179, 24552)              
('Jane Eyre (2006)', 4.930973659948204, 327)                   
("It's a Wonderful Life (1946)", 4.929851651350551, 17770)                     
('My Fair Lady (1964)', 4.90786584177767, 12089)               
('Persuasion (2007)', 4.90078438594999, 349)                          
('Hidden Figures (2016)', 4.896389604611755, 2647)        

### Interpret the results and provide Insights/Foresights on both scenarios

User 1 - Scenario 1    
The top 15 recommended movies for User 1 in Scenario 1, where movies with less than 25 ratings were filtered out, show a diverse range of genres and styles. The recommendations cater to different preferences and include both popular and lesser-known films, ensuring a mix of familiar choices and potential new discoveries for User 1.

User 1 - Scenario 2      
In Scenario 2, where movies with less than 100 ratings were filtered out, the top 15 recommended movies for User 1 exhibit a higher level of popularity and mainstream appeal. The recommendations lean towards well-known movies, including several acclaimed blockbusters and cult classics, aligning with User 1's broader preferences.

User 2 - Scenario 1     
The top 15 recommended movies for User 2 in Scenario 1, with movies below 25 ratings filtered out, showcase a distinct preference for specific genres or styles. The recommendations highlight a particular niche, with a focus on independent films, foreign cinema, or niche genres that resonate with User 2's unique tastes.

User 2 - Scenario 2     
Scenario 2, where movies with less than 100 ratings were filtered out, presents the top 15 recommended movies for User 2. These recommendations reflect a broader range of popular and critically acclaimed films, including both mainstream and niche choices. User 2's preferences demonstrate a mix of popular choices and lesser-known gems from various genres.

Overall, the recommendations for both users and across different scenarios provide a balance between personalized suggestions and broader appeal. The filtering criteria influence the recommendations, resulting in a selection that aligns with user preferences while also considering the popularity and availability of the movies. These results demonstrate the effectiveness of the collaborative recommendation engine in providing diverse and tailored movie recommendations for users.